In [ ]:
import numpy as np
from radiomics import glszm
from pathlib import Path
import imageio
import SimpleITK as sitk
import matplotlib.pyplot as plt
from radiomics import featureextractor
import pandas as pd
import re
from docx import Document
from docx.shared import Inches
import math
import pingouin as pg
from docx.shared import Inches
from io import BytesIO
from numpy import ones, kron, mean, eye, hstack, dot, tile
from numpy.linalg import pinv

# Summary

Texture metrics were extracted from the ROIs of each image and included: 
    Gray Level Size Zone Matrix (GLSZM, N = 16), 
    Gray Level Co-occurrence Matrix (GLCM, N = 24), 
    Gray Level Dependence Matrix (GLDM, N = 14), 
    Neighborhood Gray Tone Dependence Matrix (NGTDM, N = 5),
    Gray Level Run Length Matrix (GLRLM, N =16). 

In all, there were 75 texture metrics that were quantified for each image.

Reference: pyradiomics

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor()
radiomic_features = {}
directory = Path('/Invariant_Feature/data/BraTS2020_Post')
masked_filenames = list(directory.glob('**/*_mask.jpg'))
print(len(masked_filenames))

### 1. Radiomics feature values vs. Ng
### 2. statistics for each feature (mean & sd)

In [ ]:
Ng_values = [64, 96, 128, 160, 192, 224, 256]
excel_writer = pd.ExcelWriter('/Invariant_Feature/Ng_64-256.xlsx', engine='xlsxwriter')
excel_writer1 = pd.ExcelWriter('/Invariant_Feature/Ng_64-256_Stat.xlsx', engine='xlsxwriter')

for Ng in Ng_values:
    radiomic_features = pd.DataFrame()
    for mf in masked_filenames:
        cf = list(mf.parent.glob('*_t2.jpg'))[0]
        image = imageio.v2.imread(cf)
        mask = imageio.v2.imread(mf)
        # Convert the image to grayscale
        grayscale_image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])
        grayscale_image = np.round((grayscale_image / np.max(grayscale_image)) * (Ng-1)).astype(np.uint8)
        gray_scale_value = len(np.unique(grayscale_image))

        image = sitk.GetImageFromArray(grayscale_image)
        mask = sitk.GetImageFromArray((mask[:,:,1] > 240).astype(np.uint8))
    
        result = extractor.execute(image, mask)
        path_parts = mf.parts
        penultimate_mf = path_parts[-2]
        radiomic_features[penultimate_mf] = result
    
    radiomic_features = radiomic_features.transpose()
    radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='firstorder')))]
    radiomic_features = radiomic_features[radiomic_features.columns.drop(list(radiomic_features.filter(regex='diagnostics')))]
    radiomic_features = radiomic_features.sort_index()
    sheet_name = f"Ng = {Ng}"
    radiomic_features.to_excel(excel_writer, sheet_name=sheet_name, index=True)
    
    mean_values = radiomic_features.mean()
    std_values = radiomic_features.std()
    mean_std_df = pd.DataFrame({'Mean': mean_values, 'Standard Deviation': std_values})
    mean_std_df.to_excel(excel_writer1, sheet_name=sheet_name, index=True)
    
excel_writer.save()
excel_writer1.save()

### feature plot (feature vs. Ng)

In [ ]:
mean_std_file = pd.ExcelFile('/Invariant_Feature/Ng_64-256_Stat.xlsx')

ng_values = []
mean_values = []
sd_values = []
doc = Document()
section = doc.sections[0]

for i in range(2, 77):
    for sheet_name in mean_std_file.sheet_names:
        ng = re.findall(r'\d+', sheet_name)
        if ng:
            ng = int(ng[0])  
        else:
            continue 
        
        df = mean_std_file.parse(sheet_name)
        name = df.iloc[i-2, 0]  
        mean_values.append(df.iloc[i-2, 1])
        sd_values.append(df.iloc[i-2, 2])
        ng_values.append(ng)

    plt.errorbar(ng_values, mean_values, sd_values, linestyle='None', marker='^')
    plt.title(name)
    plt.xlabel('Gray Scale Level')
    plt.ylabel('Mean')
    #plt.show()
    
    #plot_filename = f'{name}.png'
    #plt.savefig(plot_filename)
    #doc.add_picture(plot_filename, width=Inches(6))
    #plt.clf()
    
    plot_filename = f'{name}.png'
    image_stream = BytesIO()
    plt.savefig(image_stream, format='png')
    image_stream.seek(0)

    doc.add_picture(image_stream, width=Inches(6))
    plt.clf()
    
    mean_values.clear()
    ng_values.clear()
    sd_values.clear()
    
doc.save('plot_Ng[64-256].docx')

### example to show invariance

In [ ]:
mean_std_file = pd.ExcelFile('/Invariant_Feature/Ng_64-256_Stat.xlsx')

ng_values = []
mean_values = []
sd_values = []
mean_values_norm = []
sd_values_norm = []

for i in range(31, 32):
    for sheet_name in mean_std_file.sheet_names:
        ng = re.findall(r'\d+', sheet_name)
        if ng:
            ng = int(ng[0])  
        else:
            continue 
        
        df = mean_std_file.parse(sheet_name)
        name = df.iloc[i-2, 0]  
        mean_values.append(df.iloc[i-2, 1])
        sd_values.append(df.iloc[i-2, 2])
        mean_values_norm.append((df.iloc[i-2, 1])/(math.log(Ng)))
        sd_values_norm.append((df.iloc[i-2, 2])/(math.log(Ng)))
        ng_values.append(ng)

    plt.errorbar(ng_values, mean_values, sd_values, linestyle='None', marker='^', label='Non-Normalized')
    plt.errorbar(ng_values, mean_values_norm, sd_values_norm, linestyle='None', marker='^', label='Normalized')
    plt.title(name)
    plt.xlabel('Gray Scale Level')
    plt.ylabel('Mean & SD')
    plt.legend()
    plt.show()


### feature vs. Ng

In [ ]:
diff_Ngs_file = pd.ExcelFile('/Invariant_Feature/Ng_64-256.xlsx')
excel_writer2 = pd.ExcelWriter('/Invariant_Feature/feature_vs_Ng.xlsx', engine='xlsxwriter')

first_sheet_name = diff_Ngs_file.sheet_names[0]
first_df = diff_Ngs_file.parse(first_sheet_name)
num_cols = len(first_df.columns)

for col_index in range(1,num_cols):
    column_data = []

    for sheet_name in diff_Ngs_file.sheet_names:
        df = diff_Ngs_file.parse(sheet_name)
        column = df.iloc[:, col_index]
        column_data.append(column)

    column_data_df = pd.DataFrame(column_data).T
    column_headers = [64, 96, 128, 160, 192, 224, 256]
    column_data_df.columns = column_headers

    #name = first_df.columns[col_index]
    name = "_".join(first_df.columns[col_index].split("_")[1:])
    column_data_df.to_excel(excel_writer2, sheet_name=name[:31], index=False)

excel_writer2.close()

### Find invariant feature

In [ ]:
def icc(Y, icc_type='ICC(3,1)'):
    ''' Calculate intraclass correlation coefficient
    Args:
        Y: The data Y are entered as a 'table' ie. subjects are in rows and repeated
            measures in columns
        icc_type: type of ICC to calculate. (ICC(2,1), ICC(2,k), ICC(3,1), ICC(3,k)) 
    Returns:
        ICC: (np.array) intraclass correlation coefficient
    '''

    [n, k] = Y.shape
    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    # create the design matrix for the different levels
    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                                X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc  # / n (without n in SPSS results)

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        # ICC = (MSR - MSRW) / (MSR + (k-1) * MSRW)
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'ICC(2,1)' or icc_type == 'ICC(2,k)':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        if icc_type == 'ICC(2,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'ICC(3,1)' or icc_type == 'ICC(3,k)':
        # ICC(3,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error)
        if icc_type == 'ICC(3,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

In [ ]:
features_file = pd.ExcelFile('/Invariant_Feature/feature_vs_Ng.xlsx')
df_non_norm = features_file.parse('ngtdm_Strength')
df_non_norm.head()

In [ ]:
df_norm = pd.DataFrame(columns=df_non_norm.columns)
for col in df_non_norm.columns:
    header_value = df_non_norm[col].name
    df_norm[col] = df_non_norm[col]*(math.log(header_value)) 
df_norm.head()

In [ ]:
array = df_non_norm.values
ICC_tmp = icc(array)
ICC_tmp = round(ICC_tmp, 2)
print("Non-normalization ICC:", ICC_tmp)

array2 = df_norm.values
ICC_tmp2 = icc(array2)
ICC_tmp2 = round(ICC_tmp2, 2)
print("Normalization ICC:", ICC_tmp2)

### generate final invariant features

In [ ]:
formula_file = pd.ExcelFile('/Invariant_Feature/Invariant_Features_sheet.xlsx')
df_formula = pd.read_excel(formula_file, sheet_name='features')
formula_column = df_formula.iloc[:, 4]
print(formula_column)

In [ ]:
Feature_Ngs_file = pd.ExcelFile('/Invariant_Feature/Ng_64-256.xlsx')
excel_writer = pd.ExcelWriter('/Invariant_Feature/Empirical_Invariant_Feature_BraTS2020.xlsx')
df_formula = pd.read_excel(Feature_Ngs_file, sheet_name='Ng = 256')
columns_to_calculate = Feature_Ngs_file.parse(sheet_name='Ng = 256').columns[1:]

Ng = 256

for i, column in enumerate(columns_to_calculate):
    formula_number = formula_column[i] 
    if formula_number == 1:
        df_formula[column] = df_formula[column] * Ng
    elif formula_number == 2:
        df_formula[column] = df_formula[column] / Ng
    elif formula_number == 3:
        df_formula[column] = df_formula[column] / (Ng*Ng)
    elif formula_number == 4:
        df_formula[column] = df_formula[column] / (Ng*Ng*Ng)
    elif formula_number == 5:
        df_formula[column] = df_formula[column] / (math.log(Ng)) 
    elif formula_number == 6:
        df_formula[column] = df_formula[column] * (math.log(Ng)) 
    elif formula_number == 7:
        df_formula[column] = df_formula[column] / (math.log(Ng*Ng)) 
    elif formula_number == 8:
        df_formula[column] = df_formula[column] * Ng * Ng
    elif formula_number == 9:
        df_formula[column] = df_formula[column] * Ng * Ng * Ng

df_formula.to_excel(excel_writer, sheet_name='Empirical_Invariant', index=False)

excel_writer.close()